Next, we want to see whether scLEMBAS can capture the heterogeneity of cell responses upon ligand exposure. 

In [1]:
import os

import numpy as np
import pandas as pd

import scanpy as sc
from sklearn.neighbors import NearestCentroid
from scipy.spatial.distance import cdist, pdist, squareform

import torch

import matplotlib.pyplot as plt

import sys
# lembas_path = '/nobackup/users/hmbaghda/Software/LEMBAS'
lembas_path = '/nobackup/users/hmbaghda/Software/avlant_LEMBASGPU'

sclembas_path = '/home/hmbaghda/Projects/scLEMBAS/scLEMBAS'
sys.path.insert(1, os.path.join(sclembas_path))
from model.bionetwork import format_network, SignalingModel

/tmp/ipykernel_1695058/1969642366.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
n_cores = 12
os.environ["OMP_NUM_THREADS"] = str(n_cores)
os.environ["MKL_NUM_THREADS"] = str(n_cores)
os.environ["OPENBLAS_NUM_THREADS"] = str(n_cores)
os.environ["VECLIB_MAXIMUM_THREADS"] = str(n_cores)
os.environ["NUMEXPR_NUM_THREADS"] = str(n_cores)

seed = 888
data_path = '/nobackup/users/hmbaghda/scLEMBAS/analysis'

device = "cuda" if torch.cuda.is_available() else "cpu"

# Model here

Load Data:

In [3]:
# prior knowledge signaling network
net = pd.read_csv(os.path.join(lembas_path, 'data', 'macrophage-Model.tsv'), sep = '\t', index_col = False)

# ligand input and TF output
ligand_input = pd.read_csv(os.path.join(lembas_path, 'data', 'macrophage-Ligands.tsv'), sep='\t', low_memory=False, index_col=0)
tf_output = pd.read_csv(os.path.join(lembas_path, 'data', 'macrophage-TFs.tsv'), sep='\t', low_memory=False, index_col=0)

Let's see what the signaling network looks like:

In [4]:
stimulation_label = 'stimulation'
inhibition_label = 'inhibition'
weight_label = 'mode_of_action'
source_label = 'source'
target_label = 'target'

net[[source_label, target_label, stimulation_label, inhibition_label]].head()

,source,target,stimulation,inhibition
0,P49137,Q16539,0,0
1,Q16539,P49137,1,0
2,P31749,O15111,1,0
3,O15111,P19838,1,0
4,P19838,O15111,0,0


Let's format it to fit with the necessary inputs to the SignalingModel:

In [5]:
net = format_network(net, weight_label = weight_label, stimulation_label = stimulation_label, inhibition_label = inhibition_label)
net[[source_label, target_label, weight_label, stimulation_label, inhibition_label]].head()

,source,target,mode_of_action,stimulation,inhibition
0,P49137,Q16539,0.1,0,0
1,Q16539,P49137,1.0,1,0
2,P31749,O15111,1.0,1,0
3,O15111,P19838,1.0,1,0
4,P19838,O15111,0.1,0,0


Next, let's initialize the model and format the inputs/outputs for running the model:

In [6]:
training_parameters = {'targetSteps': 100, 'maxSteps': 150, 'expFactor':50, 'tolerance': 1e-5, 'leak':1e-2}
mod = SignalingModel(net = net,
                     X_in = ligand_input,
                     y_out = tf_output, 
                     projection_amplitude = 3, projection_factor = 1.2,
                     weight_label = 'mode_of_action', source_label = 'source', target_label = 'target',
                     bionet_params = training_parameters, dtype = torch.float32, device = device)

X_in = mod.df_to_tensor(mod.X_in)
y_out = mod.df_to_tensor(mod.y_out)

The ligand input, after filtering for nodes in the network, looks like this:

In [7]:
mod.X_in.head()

,Ligand_GC,Ligand_IC,Ligand_IFNb,Ligand_IFNg,Ligand_IL10,Ligand_IL13,Ligand_IL4,Ligand_LPSc,Ligand_P3C,Ligand_PGE2,Ligand_TNFa,Ligand_upLPS
CON,0,0,0,0,0,0,0,0,0,0,0,0
GC,1,0,0,0,0,0,0,0,0,0,0,0
IFNb,0,0,1,0,0,0,0,0,0,0,0,0
IFNb+TNFa+PGE2+P3C,0,0,1,0,0,0,0,0,1,1,1,0
IFNb+TNFa+PGE2+P3C+IFNg,0,0,1,1,0,0,0,0,1,1,1,0


The TF activity output, after filtering for nodes in the network, looks like this:

In [8]:
mod.y_out.head()

,O43524,O75030,P01100,P01106,P03372,P04637,P05412,P08047,P10070,P10242,...,Q07869,Q08050,Q12778,Q13127,Q13485,Q14186,Q16665,Q9H3D4,Q9NQB0,Q9UJU2
CON,0.543487,0.506355,0.288705,0.524944,0.508840,0.535930,0.456016,0.799885,0.465701,0.407298,...,0.479674,0.404891,0.409449,0.568049,0.439274,0.702048,0.450063,0.147630,0.384904,0.677774
GC,0.677927,0.617778,0.119494,0.724938,0.613868,0.796771,0.011826,0.439144,0.421594,0.677585,...,0.299406,0.103314,0.656781,0.440104,0.544396,0.482236,0.094289,0.250332,0.209280,0.637135
IFNb,0.384689,0.621748,0.109474,0.352545,0.558823,0.512618,0.187316,0.326541,0.430194,0.573846,...,0.505945,0.732870,0.513355,0.429696,0.509111,0.545478,0.131642,0.557754,0.465558,0.649616
IFNb+TNFa+PGE2+P3C,0.894344,0.618304,0.815526,0.029120,0.789124,0.627074,0.393195,0.551965,0.399580,0.498236,...,0.241678,0.735164,0.709700,0.383326,0.503909,0.150867,0.736157,0.556562,0.390066,0.232772
IFNb+TNFa+PGE2+P3C+IFNg,0.757642,0.506760,0.772328,0.033346,0.645125,0.762146,0.619820,0.420949,0.406377,0.303368,...,0.088150,0.655950,0.762460,0.409982,0.199154,0.199775,0.818583,0.677592,0.346423,0.165553


In [78]:
self = ProjectInput(mod.node_idx_map, mod.input_labels, projection_amplitude = 3, dtype = mod.dtype)

In [29]:
X_in = mod.df_to_tensor(mod.X_in)
weights = 3 * torch.ones(len(mod.input_labels), dtype=mod.dtype)
weights = weights.to(mod.device)

In [37]:
weights[3] = 5

In [30]:
weights.shape

torch.Size([12])

In [31]:
X_in.shape

torch.Size([23, 12])

In [38]:
weights*X_in

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 3., 0., 0., 0., 0., 0., 3., 3., 3., 0.],
        [0., 0., 3., 5., 0., 0., 0., 0., 3., 3., 3., 0.],
        [0., 0., 0., 5., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 5., 0., 0., 0., 0., 0., 0., 3., 0.],
        [0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0., 3.],
        [0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0.],
        [0., 3., 0., 0., 0., 0., 0., 3., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 3., 3., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 3., 0.],
        [0., 0

In [ ]:
class ProjectInput(nn.Module):
    """Generate all nodes for the signaling network and linearly scale input ligand values by learned parameters."""
    def __init__(self, node_idx_map: Dict[str, int], input_labels: np.array, projection_amplitude: Union[int, float], dtype: torch.dtype=torch.float32):
        """Initialization method for ProjectInput

        Parameters
        ----------
        node_idx_map : Dict[str, int]
            a dictionary mapping node labels (str) to the node index (float)
            generated by `SignalingModel.parse_network`
        input_labels : np.array
            names of the input nodes (ligands) from net
        projection_amplitude : Union[int, float]
            value with which to initialize learned parameters (if turn require_grad = False for this layer, this is still applied 
            simply as a constant linear scalar in each forward pass)
        dtype : torch.dtype, optional
            datatype to store values in torch, by default torch.float32
        """
        super().__init__()

        self.projection_amplitude = projection_amplitude
        self.size_out = len(node_idx_map) # number of nodes total in prior knowledge network
        self.input_node_order = torch.tensor([node_idx_map[x] for x in input_labels]) # idx representation of inputs
        weights = self.projection_amplitude * torch.ones(len(input_labels), dtype=dtype)
        self.weights = nn.Parameter(weights)
        
    def forward(self, X_in):
        """Learn the weights for the input ligands to the signaling network"""
        X_full = torch.zeros([X_in.shape[0],  self.size_out], dtype=X_in.dtype, device=X_in.device) # shape of (samples x total nodes in network)
        X_full[:, self.input_node_order] = self.weights * X_in # only modify those nodes that are part of the input (ligands)
        return X_full
    
    def L2Reg(self, L2):
        """Apply an L2 regularization to the neural network parameters"""
        projection_L2 = L2 * torch.sum(torch.square(self.weights - self.projection_amplitude))  
        return projection_L2    
    
    def set_device(self, device):
        self.input_node_order = self.input_node_order.to(device)

In [42]:
type(weights)

torch.Tensor